In [ ]:
import os
import re
import time
from datetime import date
from urllib.parse import urlparse, urljoin

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# ====== 설정 ======
LIST_URL = "https://www.hyundaicard.com/cpu/ug/CPUUG2001_08.hc"
OUT_DIR = os.path.abspath("downloads_product_manuals")
MAX_ITEMS = None  # 전체 다 받으려면 None (테스트할 땐 5 정도 넣으세요)
RETRY_PER_ITEM = 3
# ==================

def sanitize_filename(s: str) -> str:
    s = re.sub(r"<[^>]+>", "", s)
    s = re.sub(r"[\\/:*?\"<>|]", "_", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s[:100] # 파일명 너무 길면 잘림 방지

def parse_kr_date(s: str) -> date:
    try:
        parts = [p.strip() for p in s.split(".") if p.strip()]
        if len(parts) >= 3:
            return date(int(parts[0]), int(parts[1]), int(parts[2]))
    except:
        pass
    return date.today()

def get_base_url(current_url: str) -> str:
    p = urlparse(current_url)
    return f"{p.scheme}://{p.netloc}"

def unique_path(path: str) -> str:
    if not os.path.exists(path):
        return path
    base, ext = os.path.splitext(path)
    k = 2
    while True:
        cand = f"{base}_{k}{ext}"
        if not os.path.exists(cand):
            return cand
        k += 1

def close_popup_hyundai(driver):
    """팝업 닫기 (빠르고 간결하게)"""
    try:
        driver.execute_script("if(window.popup) window.popup.close('popDownload', 'btnDownload');")
    except: pass
    
    try:
        # 혹시 버튼이 남아있으면 강제 클릭
        driver.execute_script("""
            var btn = document.querySelector('#popDownload .layer_close a');
            if(btn) btn.click();
        """)
    except: pass

def wait_download_complete(download_dir, before_files, timeout=30):
    """
    가장 확실한 다운로드 체크 로직:
    1. .crdownload (임시파일)이 있으면 기다린다.
    2. 새로운 파일이 생겼고, 그게 .crdownload가 아니면 완료로 본다.
    """
    start_time = time.time()
    
    while time.time() - start_time < timeout:
        current_files = set(os.listdir(download_dir))
        new_files = current_files - before_files
        
        # 1. 임시 파일(.crdownload)이 있는지 확인 -> 있으면 다운로드 중인 것
        temp_files = [f for f in new_files if f.endswith('.crdownload')]
        if temp_files:
            time.sleep(0.5) # 아직 받는 중이니 대기
            continue
            
        # 2. 임시 파일은 없는데 새 파일이 생겼다? -> 다운로드 완료!
        if new_files:
            # 가장 최근 파일 찾기
            try:
                latest_file = max(new_files, key=lambda f: os.path.getmtime(os.path.join(download_dir, f)))
                # 파일 크기가 0보다 큰지 확인 (생성 직후 0바이트일 수 있음)
                if os.path.getsize(os.path.join(download_dir, latest_file)) > 0:
                    return latest_file
            except OSError:
                pass # 파일 처리 중 에러나면 다음 루프
                
        time.sleep(0.5)
        
    return None

def extract_latest_pdf_url(driver, base_url):
    # 팝업 내용 로딩
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#popup_content tr")))
    rows = driver.find_elements(By.CSS_SELECTOR, "#popup_content tr")
    
    candidates = []
    for row in rows:
        try:
            tds = row.find_elements(By.TAG_NAME, "td")
            if len(tds) < 2: continue
            
            d = parse_kr_date(tds[0].text.strip())
            link = row.find_element(By.CSS_SELECTOR, "a.ico_download_link")
            href = link.get_attribute("href")
            if not href: continue
            
            full_url = href if href.startswith("http") else urljoin(base_url, href)
            candidates.append((d, full_url))
        except: continue

    if not candidates:
        raise RuntimeError("PDF 링크 없음")

    return max(candidates, key=lambda x: x[0])

def main():
    if not os.path.exists(OUT_DIR):
        os.makedirs(OUT_DIR)

    options = webdriver.ChromeOptions()
    prefs = {
        "download.default_directory": OUT_DIR,
        "download.prompt_for_download": False,
        "plugins.always_open_pdf_externally": True, 
        "profile.default_content_setting_values.automatic_downloads": 1,
    }
    options.add_experimental_option("prefs", prefs)
    options.add_experimental_option("excludeSwitches", ["enable-logging"])
    
    # [속도 향상] 페이지 로딩 전략 변경 (eager: HTML만 받으면 진행)
    options.page_load_strategy = 'eager' 

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    wait = WebDriverWait(driver, 15)

    print(f"ㅇㅅㅇ: 다운로드 시작합니다! 대상 폴더: {OUT_DIR}")

    try:
        driver.get(LIST_URL)
        
        # 목록 가져오기
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a.ico_download")))
        btns = driver.find_elements(By.CSS_SELECTOR, "a.ico_download")
        
        items = []  
        for b in btns:
            items.append({
                "sqno": b.get_attribute("sqno"),
                "idxdetail": b.get_attribute("idxdetail"),
                "title": b.get_attribute("webblbdtitl") or b.text or "제목없음"
            })
            
        print(f"총 {len(items)}개 파일을 발견했습니다.")
        if MAX_ITEMS: items = items[:MAX_ITEMS]
        
        success = 0
        
        for i, item in enumerate(items, 1):
            sqno = item['sqno']
            idx = item['idxdetail']
            raw_title = item['title']
            safe_title = sanitize_filename(raw_title)
            
            print(f"\n[{i}/{len(items)}] {safe_title} 처리 중...")
            
            for attempt in range(1, RETRY_PER_ITEM + 1):
                try:
                    # 목록 페이지 복귀 체크
                    if not driver.current_url.startswith(LIST_URL):
                        driver.get(LIST_URL)
                        
                    # 다운로드 버튼이 보일 때까지 대기
                    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a.ico_download")))
                    
                    # [속도 최적화] 무조건 쉬지 않고, JS가 준비될 최소 시간만 줌
                    time.sleep(0.5) 
                    
                    # 1. 팝업 열기
                    driver.execute_script(f"getDownloadList({sqno}, '{idx}', '{raw_title}');")
                    
                    # 2. 팝업 대기 및 URL 추출
                    # popDownload가 보일 때까지 기다림
                    wait.until(EC.visibility_of_element_located((By.ID, "popDownload")))
                    
                    latest_date, pdf_url = extract_latest_pdf_url(driver, get_base_url(driver.current_url))
                    
                    # 3. 팝업 닫기 (다운로드 전 정리)
                    close_popup_hyundai(driver)
                    
                    # 4. 다운로드 실행
                    before_files = set(os.listdir(OUT_DIR))
                    driver.get(pdf_url)
                    
                    # 5. [핵심] 다운로드 완료 대기 (crdownload 감시 방식)
                    new_file = wait_download_complete(OUT_DIR, before_files, timeout=20)
                    
                    if new_file:
                        # 파일명 변경
                        date_str = latest_date.strftime('%Y%m%d')
                        final_name = f"{safe_title}_{date_str}.pdf"
                        src_path = os.path.join(OUT_DIR, new_file)
                        dst_path = unique_path(os.path.join(OUT_DIR, final_name))
                        
                        # 이름 변경 시도
                        try:
                            os.rename(src_path, dst_path)
                        except:
                            time.sleep(1) # 파일 잠금 해제 대기
                            os.rename(src_path, dst_path)
                            
                        print(f"   ✅ 성공! -> {final_name}")
                        success += 1
                        break # 성공했으니 재시도 루프 탈출!
                    else:
                        raise RuntimeError("파일 다운로드 시간 초과")

                except Exception as e:
                    print(f"   ⚠️ 재시도({attempt}) : {e}")
                    close_popup_hyundai(driver) # 에러나면 팝업 닫고 다시 시도
                    time.sleep(1)
            
            # 너무 빠르면 서버 차단 당할 수 있으니 0.5초만 숨 고르기
            time.sleep(0.5)

    finally:
        driver.quit()
        print(f"\nㅇㅅㅇ: 모든 작업 끝! 총 {success}개 성공!")

if __name__ == "__main__":
    main()


# 파일이 깨진게 절반정도 되고 69개에서 멈춤

ㅇㅅㅇ: 다운로드 시작합니다! 대상 폴더: c:\Users\Playdata\workplace\final\practice\downloads_product_manuals
총 384개 파일을 발견했습니다.

[1/384] DB손해보험 현대카드 처리 중...
   ✅ 성공! -> DB손해보험 현대카드_20250804.pdf

[2/384] 현대해상 현대카드 처리 중...
   ✅ 성공! -> 현대해상 현대카드_20250918.pdf

[3/384] 현대카드H 처리 중...
   ✅ 성공! -> 현대카드H_20250827.pdf

[4/384] 현대카드Z everyday 처리 중...
   ✅ 성공! -> 현대카드Z everyday_20250825.pdf

[5/384] 현대카드O 처리 중...
   ✅ 성공! -> 현대카드O_20250827.pdf

[6/384] 현대카드S 처리 중...
   ✅ 성공! -> 현대카드S_20250827.pdf

[7/384] 현대카드D 처리 중...
   ✅ 성공! -> 현대카드D_20250827.pdf

[8/384] 현대카드T 처리 중...
   ✅ 성공! -> 현대카드T_20250827.pdf

[9/384] 현대카드 X Cut 처리 중...
   ✅ 성공! -> 현대카드 X Cut_20250813.pdf

[10/384] 현대카드 X Save 처리 중...
   ✅ 성공! -> 현대카드 X Save_20250813.pdf

[11/384] 현대카드 ZERO Up 처리 중...
   ✅ 성공! -> 현대카드 ZERO Up_20250813.pdf

[12/384] The Centurion® Card 처리 중...
   ✅ 성공! -> The Centurion® Card_20250408.pdf

[13/384] 현대카드MY BUSINESS Summit 처리 중...
   ✅ 성공! -> 현대카드MY BUSINESS Summit_20250402.pdf

[14/384] 캐시노트 BUSINESS 현대카드 처리 중...
   ✅ 성공! 

KeyboardInterrupt: 

In [ ]:
import os
import re
import time
import random
import requests # 파일 다운로드용 (pip install requests 필요)
from datetime import date
from urllib.parse import urlparse, urljoin

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# ====== 설정 ======
LIST_URL = "https://www.hyundaicard.com/cpu/ug/CPUUG2001_08.hc"
OUT_DIR = os.path.abspath("downloads_product_manuals_final") # 폴더명 변경
MAX_ITEMS = None  # 전체 수집
RETRY_PER_ITEM = 3
# ==================

def sanitize_filename(s: str) -> str:
    s = re.sub(r"<[^>]+>", "", s)
    s = re.sub(r"[\\/:*?\"<>|]", "_", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s[:100]

def parse_kr_date(s: str) -> date:
    try:
        parts = [p.strip() for p in s.split(".") if p.strip()]
        if len(parts) >= 3:
            return date(int(parts[0]), int(parts[1]), int(parts[2]))
    except:
        pass
    return date.today()

def get_base_url(current_url: str) -> str:
    p = urlparse(current_url)
    return f"{p.scheme}://{p.netloc}"

def close_popup_hyundai(driver):
    """팝업 닫기: 꼬임 방지를 위해 JS로 강제 종료"""
    try:
        driver.execute_script("if(window.popup) window.popup.close('popDownload', 'btnDownload');")
    except: pass
    try:
        # 혹시 남아있는 닫기 버튼 처리
        driver.execute_script("var btn=document.querySelector('.layer_close a'); if(btn) btn.click();")
    except: pass

def extract_latest_pdf_info(driver, base_url):
    """팝업에서 (최신날짜, PDF URL) 추출"""
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#popup_content tr")))
    rows = driver.find_elements(By.CSS_SELECTOR, "#popup_content tr")
    
    candidates = []
    for row in rows:
        try:
            tds = row.find_elements(By.TAG_NAME, "td")
            if len(tds) < 2: continue
            
            d = parse_kr_date(tds[0].text.strip())
            link = row.find_element(By.CSS_SELECTOR, "a.ico_download_link")
            href = link.get_attribute("href")
            if not href: continue
            
            full_url = href if href.startswith("http") else urljoin(base_url, href)
            candidates.append((d, full_url))
        except: continue

    if not candidates:
        raise RuntimeError("PDF 링크를 찾을 수 없음")

    # 날짜 최신순 정렬
    return max(candidates, key=lambda x: x[0])

def download_file_with_requests(driver, url, save_path):
    """
    Selenium의 쿠키를 사용하여 Requests로 파일을 직접 다운로드.
    진짜 PDF인지 확인(Content-Type)하여 깨진 파일 방지.
    """
    # 1. Selenium의 쿠키 가져오기
    selenium_cookies = driver.get_cookies()
    session = requests.Session()
    for cookie in selenium_cookies:
        session.cookies.set(cookie['name'], cookie['value'])
    
    # 2. 헤더 설정 (봇 탐지 회피)
    headers = {
        "User-Agent": driver.execute_script("return navigator.userAgent;"),
        "Referer": LIST_URL
    }

    # 3. 요청 및 다운로드
    response = session.get(url, headers=headers, stream=True, timeout=30)
    
    # 4. 검증: 상태코드 200 OK
    if response.status_code != 200:
        raise RuntimeError(f"다운로드 실패 (HTTP {response.status_code})")
    
    # 5. 검증: Content-Type이 PDF인지 확인 (핵심!)
    content_type = response.headers.get("Content-Type", "").lower()
    if "pdf" not in content_type and "octet-stream" not in content_type:
        # PDF가 아니라 HTML 에러 페이지가 온 경우
        raise RuntimeError(f"PDF 파일이 아닙니다. (Type: {content_type}) - 차단되었거나 링크 오류")

    # 6. 파일 쓰기
    with open(save_path, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)
            
    # 7. 용량 확인 (너무 작으면 의심)
    if os.path.getsize(save_path) < 1000: # 1KB 미만
        os.remove(save_path) # 삭제
        raise RuntimeError("파일 용량이 너무 작음 (비정상 파일)")

    return True

def main():
    if not os.path.exists(OUT_DIR):
        os.makedirs(OUT_DIR)

    # 헤드리스 모드 사용 금지 (봇 탐지 가능성)
    options = webdriver.ChromeOptions()
    options.add_experimental_option("excludeSwitches", ["enable-logging"])
    
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    wait = WebDriverWait(driver, 15)

    print(f"ㅇㅅㅇ: 안전한 하이브리드 다운로더 시작! 폴더: {OUT_DIR}")

    try:
        driver.get(LIST_URL)
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a.ico_download")))
        
        # 전체 목록 스캔
        btns = driver.find_elements(By.CSS_SELECTOR, "a.ico_download")
        items = []
        for b in btns:
            items.append({
                "sqno": b.get_attribute("sqno"),
                "idxdetail": b.get_attribute("idxdetail"),
                "title": b.get_attribute("webblbdtitl") or b.text or "제목없음"
            })
            
        print(f"총 {len(items)}개의 문서를 발견했습니다.")
        if MAX_ITEMS: items = items[:MAX_ITEMS]
        
        success_cnt, fail_cnt = 0, 0
        
        for i, item in enumerate(items, 1):
            sqno = item['sqno']
            idx = item['idxdetail']
            raw_title = item['title']
            safe_title = sanitize_filename(raw_title)
            
            print(f"\n[{i}/{len(items)}] {safe_title} 처리 중...")
            
            # 이미 다운받은 파일이 있으면 건너뛰기 (이어받기 기능)
            # 날짜까지는 모르니 파일명 앞부분으로 대충 체크
            existing = [f for f in os.listdir(OUT_DIR) if f.startswith(safe_title)]
            if existing:
                print(f"   ↪ 이미 존재함 (Skip)")
                success_cnt += 1
                continue

            for attempt in range(1, RETRY_PER_ITEM + 1):
                try:
                    # 1. 목록 페이지 복귀 확인
                    if not driver.current_url.startswith(LIST_URL):
                        driver.get(LIST_URL)
                        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a.ico_download")))
                    
                    # 2. 랜덤 휴식 (서버 차단 방지 - 중요!)
                    sleep_time = random.uniform(1.5, 3.0) 
                    time.sleep(sleep_time) 

                    # 3. 팝업 열기 (JS)
                    driver.execute_script(f"getDownloadList({sqno}, '{idx}', '{raw_title}');")
                    
                    # 4. 팝업 내용 대기 및 URL 추출
                    try:
                        latest_date, pdf_url = extract_latest_pdf_info(driver, get_base_url(driver.current_url))
                    except TimeoutException:
                        # 팝업이 안 뜨는 경우 -> 해당 항목에 문제가 있음
                        print("   ⚠️ 팝업 로딩 실패 (데이터 없음?)")
                        close_popup_hyundai(driver)
                        break # 재시도해도 안 될 확률 높음 -> 다음 항목으로

                    # 5. 팝업 닫기 (URL만 알면 됨)
                    close_popup_hyundai(driver)
                    
                    # 6. [핵심] Requests로 다운로드 수행
                    final_name = f"{safe_title}_{latest_date.strftime('%Y%m%d')}.pdf"
                    final_path = os.path.join(OUT_DIR, final_name)
                    
                    print(f"   ⬇️ 다운로드 시도: {final_name}")
                    download_file_with_requests(driver, pdf_url, final_path)
                    
                    print(f"   ✅ 성공!")
                    success_cnt += 1
                    break # 성공하면 반복 탈출

                except Exception as e:
                    print(f"   ⚠️ 시도 {attempt} 실패: {e}")
                    close_popup_hyundai(driver)
                    time.sleep(2)
                    
                    if attempt == RETRY_PER_ITEM:
                        print(f"   ❌ 최종 실패 (건너뜀): {safe_title}")
                        fail_cnt += 1

    except Exception as e:
        print(f"치명적 오류 발생: {e}")
        
    finally:
        driver.quit()
        print(f"\n==== 작업 완료 ====")
        print(f"성공: {success_cnt}, 실패: {fail_cnt}")
        print(f"저장된 폴더: {OUT_DIR}")

if __name__ == "__main__":
    main()


# 다운 자체가 안됨

ㅇㅅㅇ: 안전한 하이브리드 다운로더 시작! 폴더: c:\Users\Playdata\workplace\final\practice\downloads_product_manuals_final
총 384개의 문서를 발견했습니다.

[1/384] DB손해보험 현대카드 처리 중...
   ⬇️ 다운로드 시도: DB손해보험 현대카드_20250804.pdf
   ⚠️ 시도 1 실패: HTTPSConnectionPool(host='www.hyundaicard.com', port=443): Max retries exceeded with url: /upload/card/20250804_%EA%B8%88%EC%86%8C%EB%B2%95_DB%EC%86%90%ED%95%B4%EB%B3%B4%ED%97%98%20%ED%98%84%EB%8C%80%EC%B9%B4%EB%93%9C.pdf (Caused by SSLError(SSLError(1, '[SSL: UNSAFE_LEGACY_RENEGOTIATION_DISABLED] unsafe legacy renegotiation disabled (_ssl.c:1010)')))
   ⚠️ 시도 2 실패: PDF 링크를 찾을 수 없음
   ⚠️ 시도 3 실패: PDF 링크를 찾을 수 없음
   ❌ 최종 실패 (건너뜀): DB손해보험 현대카드

[2/384] 현대해상 현대카드 처리 중...
   ⚠️ 시도 1 실패: PDF 링크를 찾을 수 없음
   ⬇️ 다운로드 시도: 현대해상 현대카드_20260102.pdf
   ⚠️ 시도 2 실패: HTTPSConnectionPool(host='www.hyundaicard.com', port=443): Max retries exceeded with url: /upload/card/250918_%EA%B8%88%EC%86%8C%EB%B2%95_%ED%98%84%EB%8C%80%ED%95%B4%EC%83%81.pdf (Caused by SSLError(SSLError(1, '[SSL: UNSAFE_LEGACY_RENE

KeyboardInterrupt: 

In [ ]:
import os
import re
import time
import random
import requests
import ssl
import urllib3
from requests.adapters import HTTPAdapter
from urllib3.poolmanager import PoolManager
from urllib3.util.ssl_ import create_urllib3_context
from datetime import date
from urllib.parse import urlparse, urljoin

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# ====== 설정 ======
LIST_URL = "https://www.hyundaicard.com/cpu/ug/CPUUG2001_08.hc"
OUT_DIR = os.path.abspath("downloads_final_completed") # 폴더명
MAX_ITEMS = None  # 전체 수집
RETRY_PER_ITEM = 3
# ==================

# 불필요한 SSL 경고 메시지 숨기기
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# ==============================================================================
# [핵심 해결책] Legacy SSL(구형 서버) 접속을 허용하는 커스텀 어댑터
# ==============================================================================
class LegacySSLAdapter(HTTPAdapter):
    """
    현대카드처럼 구형 SSL 방식을 사용하는 서버에 접속하기 위해
    OpenSSL의 보안 옵션을 강제로 낮추는 어댑터입니다.
    """
    def init_poolmanager(self, connections, maxsize, block=False):
        # SSL 컨텍스트 생성
        ctx = create_urllib3_context()
        # "Legacy Renegotiation" 허용 (이게 핵심!)
        ctx.options |= 0x4  # OP_LEGACY_SERVER_CONNECT
        ctx.check_hostname = False
        ctx.verify_mode = ssl.CERT_NONE
        
        self.poolmanager = PoolManager(
            num_pools=connections,
            maxsize=maxsize,
            block=block,
            ssl_context=ctx
        )

def sanitize_filename(s: str) -> str:
    s = re.sub(r"<[^>]+>", "", s)
    s = re.sub(r"[\\/:*?\"<>|]", "_", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s[:100]

def parse_kr_date(s: str) -> date:
    try:
        parts = [p.strip() for p in s.split(".") if p.strip()]
        if len(parts) >= 3:
            return date(int(parts[0]), int(parts[1]), int(parts[2]))
    except:
        pass
    return date.today()

def get_base_url(current_url: str) -> str:
    p = urlparse(current_url)
    return f"{p.scheme}://{p.netloc}"

def close_popup_hyundai(driver):
    try:
        driver.execute_script("if(window.popup) window.popup.close('popDownload', 'btnDownload');")
    except: pass
    try:
        driver.execute_script("var btn=document.querySelector('.layer_close a'); if(btn) btn.click();")
    except: pass

def extract_latest_pdf_info(driver, base_url):
    """팝업에서 (최신날짜, PDF URL) 추출"""
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#popup_content tr")))
    rows = driver.find_elements(By.CSS_SELECTOR, "#popup_content tr")
    
    candidates = []
    for row in rows:
        try:
            tds = row.find_elements(By.TAG_NAME, "td")
            if len(tds) < 2: continue
            
            d_text = tds[0].text.strip()
            # 날짜 파싱
            try:
                d = parse_kr_date(d_text)
            except:
                continue

            link = row.find_element(By.CSS_SELECTOR, "a.ico_download_link")
            href = link.get_attribute("href")
            if not href: continue
            
            full_url = href if href.startswith("http") else urljoin(base_url, href)
            candidates.append((d, full_url))
        except: continue

    if not candidates:
        raise RuntimeError("PDF 링크를 찾을 수 없음 (목록 비어있음)")

    return max(candidates, key=lambda x: x[0])

def download_file_with_requests(driver, url, save_path):
    """
    커스텀 SSL 어댑터를 장착한 Requests로 다운로드
    """
    session = requests.Session()
    
    # ★ 여기서 커스텀 어댑터를 장착! (현대카드 서버 뚫기)
    adapter = LegacySSLAdapter()
    session.mount("https://", adapter)
    
    # 쿠키 복사 (Selenium -> Requests)
    for cookie in driver.get_cookies():
        session.cookies.set(cookie['name'], cookie['value'])
    
    headers = {
        "User-Agent": driver.execute_script("return navigator.userAgent;"),
        "Referer": LIST_URL
    }

    # verify=False 옵션 추가 (SSL 인증서 검사 무시)
    response = session.get(url, headers=headers, stream=True, timeout=30, verify=False)
    
    if response.status_code != 200:
        raise RuntimeError(f"HTTP 에러 {response.status_code}")
    
    content_type = response.headers.get("Content-Type", "").lower()
    if "pdf" not in content_type and "octet-stream" not in content_type:
        raise RuntimeError(f"PDF 아님 (Type: {content_type}) - 차단됨?")

    with open(save_path, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)
            
    if os.path.getsize(save_path) < 1000:
        os.remove(save_path)
        raise RuntimeError("파일 용량 너무 작음 (손상 의심)")

    return True

def main():
    if not os.path.exists(OUT_DIR):
        os.makedirs(OUT_DIR)

    options = webdriver.ChromeOptions()
    options.add_experimental_option("excludeSwitches", ["enable-logging"])
    
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    wait = WebDriverWait(driver, 15)

    print(f"ㅇㅅㅇ: 무적 다운로더 가동! (SSL 우회 모드)")
    print(f"저장 경로: {OUT_DIR}")

    try:
        driver.get(LIST_URL)
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a.ico_download")))
        
        btns = driver.find_elements(By.CSS_SELECTOR, "a.ico_download")
        items = []
        for b in btns:
            items.append({
                "sqno": b.get_attribute("sqno"),
                "idxdetail": b.get_attribute("idxdetail"),
                "title": b.get_attribute("webblbdtitl") or b.text or "제목없음"
            })
            
        print(f"총 {len(items)}개의 문서 발견.")
        if MAX_ITEMS: items = items[:MAX_ITEMS]
        
        success_cnt, fail_cnt = 0, 0
        
        for i, item in enumerate(items, 1):
            sqno = item['sqno']
            idx = item['idxdetail']
            raw_title = item['title']
            safe_title = sanitize_filename(raw_title)
            
            print(f"\n[{i}/{len(items)}] {safe_title}")
            
            # 이어받기 체크
            existing = [f for f in os.listdir(OUT_DIR) if f.startswith(safe_title)]
            if existing:
                print(f"   ↪ 이미 있음 (Skip)")
                success_cnt += 1
                continue

            for attempt in range(1, RETRY_PER_ITEM + 1):
                try:
                    # 목록 페이지 복귀 확인
                    if not driver.current_url.startswith(LIST_URL):
                        driver.get(LIST_URL)
                        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a.ico_download")))
                    
                    time.sleep(random.uniform(1.0, 2.0)) # 랜덤 대기

                    # 팝업 열기
                    driver.execute_script(f"getDownloadList({sqno}, '{idx}', '{raw_title}');")
                    
                    try:
                        latest_date, pdf_url = extract_latest_pdf_info(driver, get_base_url(driver.current_url))
                    except:
                        # 팝업 안 뜸 -> 닫고 재시도
                        close_popup_hyundai(driver)
                        raise RuntimeError("팝업 로딩 실패")

                    close_popup_hyundai(driver)
                    
                    # 다운로드 수행 (SSL 우회 적용됨)
                    final_name = f"{safe_title}_{latest_date.strftime('%Y%m%d')}.pdf"
                    final_path = os.path.join(OUT_DIR, final_name)
                    
                    download_file_with_requests(driver, pdf_url, final_path)
                    
                    print(f"   ✅ 성공!")
                    success_cnt += 1
                    break 

                except Exception as e:
                    print(f"   ⚠️ 시도 {attempt} 에러: {e}")
                    close_popup_hyundai(driver)
                    time.sleep(1)
                    
                    if attempt == RETRY_PER_ITEM:
                        print(f"   ❌ 최종 실패: {safe_title}")
                        fail_cnt += 1

    except Exception as e:
        print(f"치명적 오류: {e}")
        
    finally:
        driver.quit()
        print(f"\n==== 작업 끝 ====")
        print(f"성공: {success_cnt}, 실패: {fail_cnt}")

if __name__ == "__main__":
    main()


# 실패를 너무 많이함

ㅇㅅㅇ: 무적 다운로더 가동! (SSL 우회 모드)
저장 경로: c:\Users\Playdata\workplace\final\practice\downloads_final_completed
총 384개의 문서 발견.

[1/384] DB손해보험 현대카드
   ✅ 성공!

[2/384] 현대해상 현대카드
   ⚠️ 시도 1 에러: 팝업 로딩 실패
   ⚠️ 시도 2 에러: 팝업 로딩 실패
   ⚠️ 시도 3 에러: 팝업 로딩 실패
   ❌ 최종 실패: 현대해상 현대카드

[3/384] 현대카드H
   ⚠️ 시도 1 에러: 팝업 로딩 실패
   ⚠️ 시도 2 에러: 팝업 로딩 실패
   ✅ 성공!

[4/384] 현대카드Z everyday
   ⚠️ 시도 1 에러: 팝업 로딩 실패
   ✅ 성공!

[5/384] 현대카드O
   ⚠️ 시도 1 에러: 팝업 로딩 실패
   ⚠️ 시도 2 에러: 팝업 로딩 실패
   ✅ 성공!

[6/384] 현대카드S
   ⚠️ 시도 1 에러: 팝업 로딩 실패
   ⚠️ 시도 2 에러: 팝업 로딩 실패
   ⚠️ 시도 3 에러: 팝업 로딩 실패
   ❌ 최종 실패: 현대카드S

[7/384] 현대카드D
   ✅ 성공!

[8/384] 현대카드T
   ⚠️ 시도 1 에러: 팝업 로딩 실패
   ⚠️ 시도 2 에러: 팝업 로딩 실패
   ⚠️ 시도 3 에러: 팝업 로딩 실패
   ❌ 최종 실패: 현대카드T

[9/384] 현대카드 X Cut
   ⚠️ 시도 1 에러: 팝업 로딩 실패
   ⚠️ 시도 2 에러: 팝업 로딩 실패
   ⚠️ 시도 3 에러: 팝업 로딩 실패
   ❌ 최종 실패: 현대카드 X Cut

[10/384] 현대카드 X Save
   ⚠️ 시도 1 에러: 팝업 로딩 실패
   ⚠️ 시도 2 에러: 팝업 로딩 실패
   ⚠️ 시도 3 에러: 팝업 로딩 실패
   ❌ 최종 실패: 현대카드 X Save

[11/384] 현대카드 ZERO Up

==== 작업 끝 ====
성공: 5, 실패: 5


KeyboardInterrupt: 

In [ ]:
import os
import re
import time
import random
import requests
import ssl
import urllib3
from requests.adapters import HTTPAdapter
from urllib3.poolmanager import PoolManager
from urllib3.util.ssl_ import create_urllib3_context
from datetime import date
from urllib.parse import urlparse, urljoin

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# ====== 설정 ======
LIST_URL = "https://www.hyundaicard.com/cpu/ug/CPUUG2001_08.hc"
OUT_DIR = os.path.abspath("downloads_final_completed")
MAX_ITEMS = None
RETRY_PER_ITEM = 3
# ==================

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

class LegacySSLAdapter(HTTPAdapter):
    def init_poolmanager(self, connections, maxsize, block=False):
        ctx = create_urllib3_context()
        ctx.options |= 0x4
        ctx.check_hostname = False
        ctx.verify_mode = ssl.CERT_NONE
        self.poolmanager = PoolManager(
            num_pools=connections,
            maxsize=maxsize,
            block=block,
            ssl_context=ctx
        )

def sanitize_filename(s: str) -> str:
    s = re.sub(r"<[^>]+>", "", s)
    s = re.sub(r"[\\/:*?\"<>|]", "_", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s[:100]

def parse_kr_date(s: str) -> date:
    try:
        parts = [p.strip() for p in s.split(".") if p.strip()]
        if len(parts) >= 3:
            return date(int(parts[0]), int(parts[1]), int(parts[2]))
    except:
        pass
    return date.today()

def get_base_url(current_url: str) -> str:
    p = urlparse(current_url)
    return f"{p.scheme}://{p.netloc}"

def close_popup_hyundai(driver):
    """팝업 닫기 및 대기"""
    try:
        driver.execute_script("if(window.popup) window.popup.close('popDownload', 'btnDownload');")
        time.sleep(0.5) # 닫히는 애니메이션 대기
    except: pass
    try:
        driver.execute_script("var btn=document.querySelector('.layer_close a'); if(btn) btn.click();")
    except: pass

def extract_latest_pdf_info(driver, base_url):
    # 대기 시간 10초 -> 20초로 증가 (네트워크 느릴 때 대비)
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#popup_content tr")))
    rows = driver.find_elements(By.CSS_SELECTOR, "#popup_content tr")
    
    candidates = []
    for row in rows:
        try:
            tds = row.find_elements(By.TAG_NAME, "td")
            if len(tds) < 2: continue
            
            d = parse_kr_date(tds[0].text.strip())
            link = row.find_element(By.CSS_SELECTOR, "a.ico_download_link")
            href = link.get_attribute("href")
            if not href: continue
            
            full_url = href if href.startswith("http") else urljoin(base_url, href)
            candidates.append((d, full_url))
        except: continue

    if not candidates:
        raise RuntimeError("팝업은 떴는데 파일 목록이 없음")

    return max(candidates, key=lambda x: x[0])

def download_file_with_requests(driver, url, save_path):
    session = requests.Session()
    adapter = LegacySSLAdapter()
    session.mount("https://", adapter)
    
    for cookie in driver.get_cookies():
        session.cookies.set(cookie['name'], cookie['value'])
    
    headers = {
        "User-Agent": driver.execute_script("return navigator.userAgent;"),
        "Referer": LIST_URL
    }

    try:
        response = session.get(url, headers=headers, stream=True, timeout=60, verify=False)
        if response.status_code != 200:
            raise RuntimeError(f"HTTP {response.status_code}")
        
        content_type = response.headers.get("Content-Type", "").lower()
        if "pdf" not in content_type and "octet-stream" not in content_type:
             # 가끔 서버가 HTML 에러를 줄 때가 있음
            raise RuntimeError(f"Invalid Content-Type: {content_type}")

        with open(save_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
                
        if os.path.getsize(save_path) < 1000:
            os.remove(save_path)
            raise RuntimeError("파일 용량 미달 (다운로드 실패)")
            
    except Exception as e:
        if os.path.exists(save_path):
            os.remove(save_path)
        raise e

    return True

def main():
    if not os.path.exists(OUT_DIR):
        os.makedirs(OUT_DIR)

    options = webdriver.ChromeOptions()
    options.add_experimental_option("excludeSwitches", ["enable-logging"])
    # 페이지 로딩 전략: normal (모든 리소스 로딩 대기) -> 안정성 위주
    options.page_load_strategy = 'normal'

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    wait = WebDriverWait(driver, 20)

    print(f"ㅇㅅㅇ: 안정성 강화 버전 가동! (실패 시 새로고침 모드)")

    try:
        driver.get(LIST_URL)
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a.ico_download")))
        
        btns = driver.find_elements(By.CSS_SELECTOR, "a.ico_download")
        items = []
        for b in btns:
            items.append({
                "sqno": b.get_attribute("sqno"),
                "idxdetail": b.get_attribute("idxdetail"),
                "title": b.get_attribute("webblbdtitl") or b.text or "제목없음"
            })
            
        print(f"총 {len(items)}개 문서 발견.")
        if MAX_ITEMS: items = items[:MAX_ITEMS]
        
        success_cnt, fail_cnt = 0
        
        for i, item in enumerate(items, 1):
            sqno = item['sqno']
            idx = item['idxdetail']
            raw_title = item['title']
            safe_title = sanitize_filename(raw_title)
            
            print(f"\n[{i}/{len(items)}] {safe_title}")
            
            existing = [f for f in os.listdir(OUT_DIR) if f.startswith(safe_title)]
            if existing:
                print(f"   ↪ 이미 있음 (Skip)")
                success_cnt += 1
                continue

            for attempt in range(1, RETRY_PER_ITEM + 1):
                try:
                    # 1. 페이지 상태 확인 (URL 체크)
                    if not driver.current_url.startswith(LIST_URL):
                        driver.get(LIST_URL)
                        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a.ico_download")))
                    
                    # 2. 팝업이 닫혀있는지 확실히 확인 (중요)
                    # popDownload가 화면에 안 보일 때까지 기다림
                    try:
                        wait.until(EC.invisibility_of_element_located((By.ID, "popDownload")))
                    except:
                        # 안 닫히면 강제로 닫기 시도
                        close_popup_hyundai(driver)

                    time.sleep(1.0) # 숨 고르기

                    # 3. 팝업 열기
                    driver.execute_script(f"getDownloadList({sqno}, '{idx}', '{raw_title}');")
                    
                    # 4. 팝업 내용 로딩 및 URL 추출
                    latest_date, pdf_url = extract_latest_pdf_info(driver, get_base_url(driver.current_url))
                    
                    # 5. 팝업 닫기
                    close_popup_hyundai(driver)
                    
                    # 6. 다운로드
                    final_name = f"{safe_title}_{latest_date.strftime('%Y%m%d')}.pdf"
                    final_path = os.path.join(OUT_DIR, final_name)
                    
                    download_file_with_requests(driver, pdf_url, final_path)
                    
                    print(f"   ✅ 성공!")
                    success_cnt += 1
                    break 

                except Exception as e:
                    print(f"   ⚠️ 시도 {attempt} 에러: {e}")
                    
                    # ★ 핵심: 실패하면 무조건 새로고침 (Refresh)
                    # JS 상태가 꼬였을 때 가장 확실한 해결책
                    print("   🔄 페이지 새로고침 중...")
                    try:
                        close_popup_hyundai(driver)
                        driver.refresh()
                        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a.ico_download")))
                        time.sleep(2) # 로딩 후 안정화 시간
                    except:
                        pass
                    
                    if attempt == RETRY_PER_ITEM:
                        print(f"   ❌ 최종 실패: {safe_title}")
                        fail_cnt += 1

    except Exception as e:
        print(f"치명적 오류: {e}")
        
    finally:
        driver.quit()
        print(f"\n==== 작업 완료 ====")
        print(f"성공: {success_cnt}, 실패: {fail_cnt}")

if __name__ == "__main__":
    main()